Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [1]:
NAME = "Mauro Vargas Jr"
COLLABORATORS = ""

---

<a id='top'></a>

# CSCI 3202, Spring 2021:  Assignment 5  

Shortcuts:  [top](#top) -- [1](#p1) | [1a](#p1a) | [1b](#p1b) | [1c](#p1c) | [1d](#p1d) | [1e](#p1e) | [1f](#p1f) | [1g](#p1g) -- [2](#p2) | [2a](#p2a) | [2b](#p2b) | [2c](#p2c) | [2d](#p2d) | [2e](#p2e) 

# Assignment Overview

This assignment is an exercise in implementing and analyzing Markov Decision Processes (MDPs). Problem 1 asks you to code a solution to a specific scenario, while Problem 2 is a conceptual question which asks you to describe an MDP problem of your own design.

Here's a summary of the tasks required and the associated points:

| Problem #  | Tasks                                                  | Points  |
|:---        |:---                                                    |:---:    |
| 1a         | Code: Complete implementation of `MDP` class           | 10      |
| 1b         | Code: Implement `value_iteration` and `find_policy`    | 5       |
| 1c         | Code and create: Generate and illustrate optimal path  | 5       |
| 1d         | Written: analyze policy                                | 5       |
| 1e         | Code: adjust non-terminal rewards                      | 5       |
| 1f         | Code and write: adjust terminal rewards                | 5       |
| 1g         | Written: analyze transition model                      | 5       |
| 2a         | Written: define problem                                | 4       |
| 2b         | Written: define states                                 | 4       |
| 2c         | Written: define reward                                 | 4       |
| 2d         | Written: define actions and transition                 | 4       |
| 2e         | Written: define optimal policy                         | 4       |
| Total      |                                                        | 60      |


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
# add any imports you may need

<a id='p1'></a>
[Back to top](#top)

# Problem 1: Navigating an awkward situation with grace and poise

<img src='https://www.explainxkcd.com/wiki/images/5/5f/interaction.png' style="width: 600px;"/>

Suppose you are at a social event where you would like to avoid any interaction with a large number of the other attendees. It's not that you don't like them, it's just that you don't like *talking to* them. A few of your good friends are also in attendance, but they are tucked away in a corner. The rectangular room in which the event is being held spans gridcells at $x=1,2,\ldots, 6$ and $y=1,2,\ldots, 5$. At the eastern edge ($x=6$) of this first floor room, there is a balcony, with a 6-foot drop. If the event becomes unbearably awkward, you can jump off the balcony and run away. Of course, this might hurt a little bit, so we should incorporate this into our reward structure.

The terminal states and rewards associated with them are given in the diagram below. The states are represented as $(x,y)$ tuples. The available actions in non-terminal states include moving exactly 1 unit North (+y), South (-y), East (+x) or West (-x), although you should not include walking into walls, because that would be embarrassing in front of all these other people. Represent actions as one of 'N', 'S', 'E', or 'W'. For now, assume all non-terminal states have a default reward of -0.01, and use a discount factor of 0.99.

<img src="http://www.cs.colorado.edu/~tonyewong/home/resources/hw06_mdp.png" style="width: 400px;"/>

Use the following transition model for this decision process, if you are trying to move from state $s$ to state $s'$:
* you successfully move from $s$ to $s'$ with probability 0.6
* the remaining 0.4 probability is spread equally likely across state $s$ **and** all adjacent (N/S/E/W) states except for $s'$. Note that this does not necessarily mean that all adjacent states have 0.1, because some states do not have 4 adjacent states.

<a id='p1a'></a>
[Back to top](#top)

## (1a) - 10 points

Complete the `MDP` class below. The docstring comments provide some desired specifications. You may add additional methods or attributes, if you would like.

In [3]:
class MDP:
    def __init__(self, nrow, ncol, terminal_states, default_reward, df):
        '''Create/store the following attributes:
        self.states -- a list of all the states as (x,y) tuples
        self.terminal_states -- a dictionary with terminal state keys, and rewards as values
        self.default_reward -- the reward for being in any non-terminal state
        self.df -- discount factor
        ... and anything else you decide will be useful!
        '''
                # YOUR CODE HERE
        
        self.states = []
        self.get_states()
        self.terminal_states = terminal_states
        self.default_reward = default_reward
        self.df = df

#         raise NotImplementedError()
    def get_states(self):
        states = []
        for i in range(1,7):
            for j in range(1,6):
                self.states.append((i,j))
        return states
    
    def actions(self, state):
        '''Return a list of available actions from the given state.
        Possible actions are 'N','S','E','W'
        [None] are the actions available from a terminal state.
        '''
        list_action=[]
        x = state[0]
        y = state[1]
        if state in self.terminal_states:
            list_action.append(None)
            return list_action
        if(x, y + 1) in self.states:
            list_action.append('N')
        if(x, y - 1) in self.states:
            list_action.append('S')
        if (x + 1, y) in self.states:
            list_action.append('E')
        if (x - 1, y) in self.states:
            list_action.append('W')
        return list_action
        


        
    def reward(self, state):
        '''Return the reward for being in the given state'''
        # YOUR CODE HERE
        if self.terminal_states.get(state):
            return self.terminal_states.get(state)
        else:
            return self.default_reward

    
    def result(self, state, action):
        '''Return the resulting state (as a tuple) from doing the given
        action in the given state, without uncertainty. Uncertainty
        is incorporated into the transition method.
        state -- a tuple representing the current state
        action -- one of N, S, E or W, as a string
        '''
        
        assert action in self.actions(state), 'Error: action needs to be available in that state'
        assert state in self.states, 'Error: invalid state'
        
        
        # YOUR CODE HERE
#         raise NotImplementedError()
        x = state[0]
        y = state[1]
        if 'N' == action:
            return (x, y + 1)
        elif 'S' == action:
            return (x, y - 1)
        elif 'E' == action:
            return (x + 1, y)
        elif 'W' == action:
            return (x - 1, y)
            
    def transition(self, state, action):
        '''Return a list of (probability, next_state) associated
        with the possibilities of taking the given action from the given state.
        '''
        
        if action is None:
            # This happens for a terminal state
            return [(0, state)]
        else:
            # Not a terminal state
            length_of_actions = len(self.actions(state))
            lst_actions = self.actions(state)
            lst = []
            for i in range(0, len(lst_actions)):
#                 print("list_actions",lst_actions[i])
                if lst_actions[i] == action:
                    probability = 0.6
                    location =self.result(state,action)
                    lst.append((probability,location))
                else:
                    probability = 0.4/length_of_actions
                    location = self.result(state,lst_actions[i])
                    lst.append((probability,location))
        
        lst.append((0.4/length_of_actions, state))
                
        return lst
            
  

    def expected_utility(self, next_states, cur_util):
        '''Return the expected utility given generated list of possible 
        next states and the current utility, which is a dictionary of the form {state : utility}
        '''
        # YOUR CODE HERE
        #raise NotImplementedError()

        exp_util = []
        utility = 0
        for key in next_states:
            (prob, state) = key
            utility = cur_util.get(state)
            exp_util.append(prob*utility)
            
        new = sum(exp_util)
        
        return new

### (1a) tests

Note that these are non-exhaustive, because there is some flexibility in how the `transition` method works.

In [4]:
## BEGIN TESTS
nrow = 3
ncol = 3
default_reward = -0.2
discount = 0.5
terminal = {(1,3):-1, (1,2):2}
mdp_simple = MDP(nrow, ncol, terminal, default_reward, discount)

actions1 = set(mdp_simple.actions((2,2)))
assert (actions1 == {'N','S','E','W'}), "Expected set of actions is {'N','S','E','W'}, your code returned: %s" % actions1

actions2 = set(mdp_simple.actions((1,1)))
assert (actions2 == {'N','E'}), "Expected set of actions is {'N','E'}, your code returned: %s" % actions2

actions3 = set(mdp_simple.actions((1,2)))
assert (actions3 == {None}), "Expected set of actions is {None}, your code returned: %s" % actions3

reward1 = mdp_simple.reward((1,2))
assert (reward1 == 2), "Expected reward is 2, your code returned: %f" % reward1

reward2 = mdp_simple.reward((2,2))
assert (reward2 == -0.2), "Expected reward is -0.2, your code returned: %f" % reward2

result1 = mdp_simple.result((1,1), 'N')
assert (result1 == (1,2)), "Expected result is (1,2), your code returned: %s" % (result1,)

print("All tests passed: 5 points")
## END TESTS

All tests passed: 5 points


In [5]:
## BEGIN TESTS
# set values from problem statement
nrow = 5
ncol = 6
default_reward = -0.01
discount = 0.99
terminal = {(1,3):-1, (1,4):2, (1,5):2, (2,1):-1, (3,1):-1, (3,4):-1, (3,5):1,
            (4,3):-1, (4,4):-1, (6,1):-5, (6,2):-5, (6,3):-5, (6,4):-5, (6,5):-5}
mdp_p1 = MDP(nrow, ncol, terminal, default_reward, discount)

# Find the expected utility of walking N from (1,1):
util_old = {s : s[0]+s[1] for s in mdp_p1.states}

next_states = mdp_p1.transition((1,1), 'N')
assert (len(next_states) == 3), "Expected 3 possible next states, your code returned: %d" % len(next_states)

exp_util = mdp_p1.expected_utility(next_states, util_old)
assert (exp_util == 2.8), "Expected utility should be 2.8, your code returned %f" % exp_util

print("All tests passed: 5 points")
## END TESTS

All tests passed: 5 points


<a id='p1b'></a>
[Back to top](#top)

## (1b) - 5 points

Implement value iteration to calculate the utilities for each state.  Also implement a function that takes as arguments an `MDP` object and a dictionary of state-utility pairs (key-value) and returns a dictionary for the optimal policy.  The optimal policy dictionary should have state tuples as keys and the optimal move (None, N, S, E or W) as values.

In [6]:
def value_iteration(mdp, tol=1e-3):
    # TODO: 
    # 1. initialize utility for all states to 0
    # 2. for each state on the board
    #    2.1. calculate expected utility for each possible next state
    #    2.2. find best utility out of possible expected utilities
    #    2.3. define new utility of the state
    # 3. Repeat 2 until problem is converged
    
    empty_dict = {} #initialize utility for all states to 0 for each state on the board
    states = mdp.states
    terminal_dic = mdp.terminal_states
    i = 0
    while i < len(states):
        empty_dict.update({states[i]:0})
        i += 1
    for index, key in enumerate(empty_dict):
        if key in list(terminal_dic.keys()):
            empty_dict[key] = terminal_dic[key]  
    
    while True:
        lst_change = []
        for key in states:
            old_util = empty_dict.get(key)
            change = 0
            if key in mdp.terminal_states:
                continue
            else:
                lst_actions = mdp.actions(key)
                lst_utl = []
                max_utl = 0
                for action in lst_actions:
                    lst_probability_next_states = mdp.transition(key,action)

                    utl_value = mdp.expected_utility(lst_probability_next_states, empty_dict)

                    lst_utl.append(mdp.reward(key) + mdp.df*utl_value)     
            max_utl = max(lst_utl)
            empty_dict[key] = max_utl

            change = old_util - max_utl
            lst_change.append(change)
            
        if max(lst_change) == 0:
            break

    return empty_dict


    
    

    # YOUR CODE HERE
#     raise NotImplementedError()

def find_policy(mdp, utility):
    '''Return a dictionary containing the best policy for each state s,
    of the form {s : s_policy}
    '''
    best_policy_dict = {}
    
    for key in utility:
        x = key[0]
        y = key[1]
        lst_actions = mdp.actions(key)
        policy_dict = {}
        for action in lst_actions:
            if action == 'N':
                policy_dict[action] = utility.get((x, y+1))
            elif action == 'S':
                policy_dict[action] = utility.get((x, y-1))
            elif action == 'E':
                policy_dict[action] = utility.get((x+1, y))
            elif action == 'W':
                policy_dict[action] = utility.get((x-1, y))
                
        current_max = -math.inf
        best_action = ""
        for i in policy_dict:
            if policy_dict.get(i) > current_max:
                current_max = policy_dict.get(i)
                best_action = i
                
        best_policy_dict[key] = best_action
#     print(best_policy_dict)        
    return best_policy_dict
    # YOUR CODE HERE
#     raise NotImplementedError                                       

### (1b) tests

In [7]:
## BEGIN TESTS
utility = value_iteration(mdp_p1, tol=1e-6)
policy = find_policy(mdp_p1, utility)

util1 = utility[(1,5)]
assert (util1 == 2), "Expected utility of 2, your code returned %f" % util1

util2 = utility[(6,1)]
assert (util2 == -5), "Expected utility of -5, your code returned %f" % util2

util3 = round(utility[(2,5)],2)
assert (util3 == 1.74), "Expected utility of 1.74, your code returned %f" % util3

util4 = round(utility[(5,3)],2)
assert (util4 == -1.39), "Expected utility of -1.39, your code returned %f" % util4

policy1 = policy[(2,4)]
assert (policy1 == 'W'), "Expected policy is 'W', your code returned %s" % policy1

policy2 = policy[(1,1)]
assert (policy2 == 'N'), "Expected policy is 'N', your code returned %s" % policy2

print("Passed test cases: 5 points")
## END TESTS

Passed test cases: 5 points


<a id='p1c'></a>
[Back to top](#top)

## (1c) - 5 points

If we enter the room at $s_0$, what is the optimal path for us to follow? Complete the following function to generate the sequence of states along the path. If we start in state $s_0$, then your output should be in the form $[s_0, s_1, s_2, ... , s_{term}]$ where $s_{term}$ is a terminal state. Set your tolerance for value iteration to be $10^{-6}$

Additionally, create a graphic to illustrate this policy pathway, either by generating a plot in Python or by uploading a hand-drawn image and including a link to it below.

In [8]:
def find_optimal_path(mdp, state):
    '''Generate list of states visited along the optimal path given an MDP
    instance and the starting state
    '''
    util = value_iteration(mdp, tol=1e-6)
    policy = find_policy(mdp,utility)
    lst_path = []
    lst_path.append(state)
    while True:
        action = policy[state]
        if action == '':
            break
        else:
            state = mdp.result(state,action)
            lst_path.append(state)
    print(lst_path)
    return lst_path
    # YOUR CODE HERE
    # raise NotImplementedError()

Put a link to your graphic below for the path from (5,3). You can find an [example image here](http://www.cs.colorado.edu/~tonyewong/home/resources/hw06_mdp_path.png) with the optimal path starting from (5,1). **Please include a link rather than attaching a file**. This can be a link to your file in Google Drive, with the permissions set to public. The graders will not ask for permissions! The syntax for including a link in markdown is `[link text](url.com/to/your/image)`

YOUR ANSWER HERE: https://drive.google.com/file/d/1A4MfKrPNZIhIHXLLWC008v8CI_pAISim/view?usp=sharing

### (1c) tests

In [9]:
## BEGIN TESTS
path1 = find_optimal_path(mdp_p1, (5,3))
assert (path1 == [(5, 3), (5, 2), (4, 2), (3, 2), (2, 2), (2, 3), (2, 4), (1, 4)]), "The optimal path is [(5, 3), (5, 2), (4, 2), (3, 2), (2, 2), (2, 3), (2, 4), (1, 4)], your code generated %s" % path1

path2 = find_optimal_path(mdp_p1, (5,1))
assert (path2 == [(5, 1), (4, 1), (4, 2), (3, 2), (2, 2), (2, 3), (2, 4), (1, 4)]), "The optimal path is [(5, 1), (4, 1), (4, 2), (3, 2), (2, 2), (2, 3), (2, 4), (1, 4)], your code generated %s" % path2

path3 = find_optimal_path(mdp_p1, (1,1))
assert (path3 == [(1, 1), (1, 2), (2, 2), (2, 3), (2, 4), (1, 4)]), "The optimal path is [(1, 1), (1, 2), (2, 2), (2, 3), (2, 4), (1, 4)], your code generated %s" % path3

print("Passed tests: 3 points")
## END TESTS

[(5, 3), (5, 2), (4, 2), (3, 2), (2, 2), (2, 3), (2, 4), (1, 4)]
[(5, 1), (4, 1), (4, 2), (3, 2), (2, 2), (2, 3), (2, 4), (1, 4)]
[(1, 1), (1, 2), (2, 2), (2, 3), (2, 4), (1, 4)]
Passed tests: 3 points


<a id='p1d'></a>
[Back to top](#top)

## (1d) - 5 points

From (3,2) the optimal move is to walk West. If we are trying to go talk to our friends in the Northwest corner, why would we rather do this than walk North first, then West?

YOUR ANSWER HERE: If you go north the agent have a higher probability of landing on a negative terminal state like (3,4) or (4,3). Thats way its a better choice to go west. 

<a id='p1e'></a>
[Back to top](#top)

## (1e) - 5 points

How painfully awkward do you need to set the default reward for non-terminal states before the optimal move at (5,1) becomes jumping off the balcony immediately and running away? Implement the following function which returns the reward where the policy for (5,1) is to jump off the balcony.

In [10]:
def find_non_terminal_reward():
    for reward in np.arange(-0.01, -3, -0.01):

        # YOUR CODE HERE
        raise NotImplementedError()


### (1e) tests

In [11]:
reward1 = find_non_terminal_reward()
assert (reward1 == -2.09), "The expected reward is -2.09, your code returned %f" % reward1

print("Test passed: 5 points")

NotImplementedError: 

<a id='p1f'></a>
[Back to top](#top)

## (1f) - 5 points

In **1e** we assumed a certain level of loss (negative reward) just for being present.  But a more realistic approach might be to instead change the reward structure for the terminal states. Consider the terminal states with -1 reward in the default model. Let $R^*$ denote the reward associated with these states. How low does $R^*$ need to be in order for us to immediately jump off the balcony and run away? Use the default non-terminal state reward of -0.01. Implement the following function to return the value of $R^*$ which leads to a policy of jumping off the balcony at (5,1). Write a few sentences interpreting your result.

In [ ]:
def find_terminal_reward():
    
    for rstar in np.arange(-6, -12, -0.01):
        # TODO:
        # 1. set the reward of the terminal nodes 
        # 2. define MDP with appropriate parameters
        # 3. find policy for state (5,1)
        # 4. return R* if the policy for (5,1) is to jump off the balcony

        # YOUR CODE HERE
        raise NotImplementedError()

### (1f) tests

In [ ]:
reward1 = round(find_terminal_reward(),2)
assert (reward1 == -11.39), "Expected reward is -11.39, your code returned %f" % reward1

print("Passed test: 3 points")

Write a few sentences with your interpretation here:

YOUR ANSWER HERE

<a id='p1g'></a>
[Back to top](#top)

## (1g) - 5 points

Given the problem context, write a few sentences about why this is or is not an appropriate transition model. Include an interpretation of the terminal states.

YOUR ANSWER HERE

---
<a id='p2'></a>
[Back to top](#top)

# Problem 2: Define your very own MDP

For this problem, you do not need to write any code, but rather communicate your ideas clearly using complete sentences and descriptions of the concepts the questions ask about. You can, of course, include some pseudocode if it helps, but that is not strictly necessary.


<a id='p2a'></a>

## (2a) - 4 points

Describe something you think would be interesting to model using a Markov decision process.  Be **creative** - do not use any examples from your homework, class, or the textbook, and if you are working with other students, please **come up with your own example**. There are so, SO many possible answers!

Honey Bee World - is a 3 x 4 grid with a bee as an agent. The bee's goal is to reach the hive in the top right corner.

<a id='p2b'></a>

## (2b) - 4 points

What are the states associated with your MDP? Include a discussion of terminal/non-terminal states.

The grid world is filled with grass states, fire states and a hive goal state. Terminal states are the hive and the fire. Non Terminal states are the flower and grass states.



<a id='p2c'></a>

## (2c) - 4 points

What is the reward structure associated with your MDP?  Include a discussion of terminal/non-terminal states.

### Reward Model
- moveReward = -5 // For every action the bee moves
- hiveReward = +100// terminal state
- fire = -50 // terminal state
- flower = +5  // extra reward

<a id='p2d'></a>

## (2d) - 4 points

What are the actions and transition model associated with your MDP?

Actions
- up
- down
- left
- right

Transition model:

The Bee start out in state (1,1). It can move to "up" with a probability of 0.4 or move to the right with a prob 0.2. 

<a id='p2e'></a>

## (2e) - 4 points

Interpret what an optimal policy represents in the context of your particular MDP.

The bee will choice the optimal path by collecting as many flowers as possible before reaching the hive goal state and with out landing on fire terminal states.